# Practical Machine Learning and Deep Learning

# Lab 9 Graph Classification Task

In this lesson you will implement graph classifier using a graph neural network (GNN).

Graph classification is an important problem with applications across many fields, such as bioinformatics, chemoinformatics, social network analysis, urban computing, and cybersecurity. Applying graph neural networks to this problem has been a popular approach recently. This can be seen in the following research references:

[Ying et al., 2018](https://arxiv.org/abs/1806.08804),
[Cangea et al., 2018](https://arxiv.org/abs/1811.01287),
[Knyazev et al., 2018](https://arxiv.org/abs/1811.09595),
[Bianchi et al., 2019](https://arxiv.org/abs/1901.01343),
[Liao et al., 2019](https://arxiv.org/abs/1901.01484),
[Gao et al., 2019](https://openreview.net/forum?id=HJePRoAct7)

## Goal

Your goal is to understand and implement a graph neural network for graph classification and apply optimization techniques to improve model performance. You will also learn how to handle graph-structured data and train models efficiently using these advanced methods.

## Key concepts

1. Graph Neural Networks (GNNs):
- Message Passing
- Graph Convolution
- Pooling Mechanisms
2. Optimization Techniques:
- Learning Rate Scheduling
- Gradient Clipping
- Regularization (Dropout, L2 Regularization)
3. Efficient Training:
- Batch Normalization
- Mixed Precision Training
- Data Augmentation for Graph Data

## Steps

1. Data Preparation:
  - Load and preprocess the dataset
  - Perform data augmentation to enhance training data diversity
2. Model Architecture:
  - Implement a GNN architecture suitable for graph classification
  - Utilize message passing and graph convolution layers
  - Integrate pooling mechanisms to handle variable-sized graphs
3. Optimization:
  - Apply learning rate scheduling and gradient clipping during training
  - Use regularization techniques to prevent overfitting
4. Training and Evaluation:
  - Train the model on the training dataset
  - Evaluate model performance on the validation dataset
  - Fine-tune the model using the test dataset
5. Submission:
  - Generate predictions for the test set
  - Create a submission.csv file with the predicted labels

## Prerequisites

For working with graphs we will use library `dgl`. It offer a lot of methods and useful structures for working with graph structures alongside the `pytorch` support.

In [ ]:
!pip install  dgl -f https://data.dgl.ai/wheels/torch-2.3/cu121/repo.html -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.3/309.3 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 21.9 MB/s eta 0:00:00


In [ ]:
import dgl
import torch
import torch.nn as nn
import torch.nn.functional as F
from dgl.nn.pytorch import GraphConv
import pandas as pd
from dgl import DGLGraph, graph as graph_constructor, batch as construct_graph_batch
from torch.utils.data import random_split, DataLoader, Dataset
from torch import Tensor
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
import dgl.function as fn
import warnings
warnings.filterwarnings('ignore')
from tqdm.notebook import tqdm
from typing import Tuple, List
import os
os.environ['DGLBACKEND'] = 'pytorch'
import unittest

DGL backend not selected or invalid.  Assuming PyTorch for now.


Setting the default backend to "pytorch". You can change it in the ~/.dgl/config.json file or export the DGLBACKEND environment variable.  Valid options are: pytorch, mxnet, tensorflow (all lowercase)


/usr/local/lib/python3.10/dist-packages/torchdata/datapipes/__init__.py:18: UserWarning: 
################################################################################
WARNING!
The 'datapipes', 'dataloader2' modules are deprecated and will be removed in a
future torchdata release! Please see https://github.com/pytorch/data/issues/1196
to learn more and leave feedback.
################################################################################

  deprecation_warning()


## Understanding DGL Graph Format

The Deep Graph Library (DGL) provides an intuitive way to handle graph-structured data. Here is a basic rundown of the format and its components:

1. **Nodes and Edges**: In DGL, a graph is defined by nodes and edges. Each node and edge can have associated features.
2. **Graph Construction**: A graph can be constructed using `dgl.graph` or `dgl.heterograph` for heterogeneous graphs.
3. **Node and Edge Features**: Features can be added to nodes and edges using tensors.

### Example

In [ ]:
# Define a graph with 3 nodes and 3 edges
u, v = torch.tensor([0, 1, 2]), torch.tensor([1, 2, 0])
graph = dgl.graph((u, v))

# Adding node features
graph.ndata['feat'] = torch.randn(3, 5)  # 3 nodes with 5-dimensional features

# Adding edge features
graph.edata['weight'] = torch.randn(3, 4)  # 3 edges with 4-dimensional features

print(graph)

Graph(num_nodes=3, num_edges=3,
      ndata_schemes={'feat': Scheme(shape=(5,), dtype=torch.float32)}
      edata_schemes={'weight': Scheme(shape=(4,), dtype=torch.float32)})


This snippet creates a graph with 3 nodes and 3 edges, and assigns random features to the nodes and edges.

### Parameters:
- ndata (Node Data): Dictionary for storing node features.
- edata (Edge Data): Dictionary for storing edge features.

Graph Neural Networks (GNNs): Training and Operations
Graph Neural Networks (GNNs) are a class of neural networks that operate on graph-structured data. They extend the concept of convolution from grids (images) to graphs.

### Key Concepts:
#### Message Passing:

- Nodes in a graph send messages to their neighbors. Each node aggregates the received messages to update its state.

#### Graph Convolution:

- Similar to convolution in CNNs but operates on nodes and their neighbors. Each node's new representation is computed based on its own features and the features of its neighbors.

### Example: Message Passing

In [ ]:
def message_func(edges):
    return {'msg': edges.src['feat']}

def reduce_func(nodes):
    return {'h': torch.sum(nodes.mailbox['msg'], dim=1)}

graph.update_all(message_func, reduce_func)

In this example, message_func sends node features to neighbors, and reduce_func aggregates them by summing.

### Example: Graph Convolution Layer#

In [ ]:
class GCNLayer(nn.Module):
    def __init__(self, in_feats, out_feats):
        super(GCNLayer, self).__init__()
        self.conv = GraphConv(in_feats, out_feats)

    def forward(self, g, inputs):
        h = self.conv(g, inputs)
        return F.relu(h)

# Instantiate and use the GCN layer
gcn_layer = GCNLayer(5, 2)
inputs = torch.randn(3, 5)  # 3 nodes with 5-dimensional features
outputs = gcn_layer(graph, inputs)
print(outputs)

tensor([[0.3117, 0.0000],
        [0.0000, 1.2879],
        [1.0742, 1.7950]], grad_fn=<ReluBackward0>)


##Data Description and the Role of GNNs
### Data Format
The data consists of graphs, each represented with nodes and edges. Each node and edge can have associated features which are crucial for tasks like classification or regression.

### Why Use GNNs?
GNNs leverage the graph structure to capture the relationships between nodes, which traditional neural networks can't. This is particularly useful in tasks where the data is naturally represented as a graph, such as:

- Social networks
- Molecular structures
- Citation networks
### Example Data Relationships:
- Nodes: Represent entities (e.g., users, atoms).
- Edges: Represent relationships (e.g., friendships, chemical bonds).
- Node Features: Attributes of entities (e.g., user interests, atom types).
- Edge Features: Attributes of relationships (e.g., interaction strength, bond type).

Using GNNs allows for efficient and effective learning from such structured data by exploiting the connectivity patterns and feature information.

## Data reading and preprocessing

First, let's create dataset class for our task. This is the same dataset as for any other classification tasks.

---

## EXERCISE 1:
Create the following functions:

1. Initialization: Implement the __init__ method, which takes the path to the CSV file and calls a method to read the data.

2. Read Data: Implement the _read_data method, which:

- Reads the CSV file into a DataFrame.
- Creates graphs and labels from the data.
- Stores the graphs and labels in the corresponding lists.
- **Note:** Use the _create_graph method to create graphs. The graph format should conform to DGL.

3. Create Graph:
- Implement the _create_graph method, which takes the number of nodes, number of edges, lists of source and target nodes of edges, and returns a DGLGraph object.

4. Data Access Methods:

- Implement the __getitem__ method, which returns a graph and label by index.
- Implement the __len__ method, which returns the number of graphs in the dataset.

In [ ]:
class GraphClassificationDataset(Dataset):
    def __init__(self, csv_path):
        super().__init__()

        self.data_path = csv_path
        self._read_data()

    def _read_data(self) -> Tuple[List[DGLGraph], List[int]]:
        # type code here

    def _create_graph(self, num_nodes: int, num_edges: int, edges_from: List[int], edges_to: List[int]) -> DGLGraph:
        # type code here

    def __getitem__(self, index) -> Tuple[DGLGraph, int]:
        # type code here

    def __len__(self):
        # type code

In [ ]:
train_set_raw = GraphClassificationDataset('graph_classification_train.csv')
test_dataset = GraphClassificationDataset('graph_classification_test.csv')

In [ ]:
assert len(train_set_raw)==800
assert len(test_dataset)==200
print("Test cases passed")

Test cases passed


In [ ]:
# Set percentage of data to use as a training subset
train_ratio = 0.8

train_size = int(train_ratio * len(train_set_raw))
val_size = len(train_set_raw) - train_size

train_dataset, val_dataset = random_split(train_set_raw, (train_size, val_size))

In [ ]:
def create_test_csv(file_path):
    data = {
        'graph_id': [1, 2, 3],
        'num_nodes': [3, 4, 5],
        'num_edges': [2, 3, 4],
        'edges_from': ['0 1', '0 1 2', '0 1 2 3'],
        'edges_to': ['1 2', '1 2 3', '1 2 3 4'],
        'label': [1, 0, 1]
    }
    df = pd.DataFrame(data)
    df.set_index('graph_id', inplace=True)
    df.to_csv(file_path)


In [ ]:
class TestGraphClassificationDataset(unittest.TestCase):
    @classmethod
    def setUpClass(cls):
        cls.csv_path = 'test_graphs.csv'
        create_test_csv(cls.csv_path)
        cls.dataset = GraphClassificationDataset(cls.csv_path)

    def test_length(self):
        self.assertEqual(len(self.dataset), 3, "Dataset length mismatch")

    def test_first_item(self):
        graph, label = self.dataset[0]
        self.assertIsInstance(graph, DGLGraph, "First item is not a DGLGraph")
        self.assertEqual(label, 1, "First label is incorrect")
        self.assertEqual(graph.number_of_nodes(), 3, "Number of nodes in the first graph is incorrect")
        self.assertEqual(graph.number_of_edges(), 2, "Number of edges in the first graph is incorrect")

    def test_second_item(self):
        graph, label = self.dataset[1]
        self.assertIsInstance(graph, DGLGraph, "Second item is not a DGLGraph")
        self.assertEqual(label, 0, "Second label is incorrect")
        self.assertEqual(graph.number_of_nodes(), 4, "Number of nodes in the second graph is incorrect")
        self.assertEqual(graph.number_of_edges(), 3, "Number of edges in the second graph is incorrect")

    def test_third_item(self):
        graph, label = self.dataset[2]
        self.assertIsInstance(graph, DGLGraph, "Third item is not a DGLGraph")
        self.assertEqual(label, 1, "Third label is incorrect")
        self.assertEqual(graph.number_of_nodes(), 5, "Number of nodes in the third graph is incorrect")
        self.assertEqual(graph.number_of_edges(), 4, "Number of edges in the third graph is incorrect")

    def test_item_retrieval(self):
        for i in range(len(self.dataset)):
            graph, label = self.dataset[i]
            self.assertIsInstance(graph, DGLGraph, f"Item {i} is not a DGLGraph")
            self.assertIsInstance(label, int, f"Label {i} is not an integer")

    @classmethod
    def tearDownClass(cls):
        os.remove(cls.csv_path)

# Run the tests
if __name__ == '__main__':
    unittest.main(argv=[''], exit=False)


.....
----------------------------------------------------------------------
Ran 5 tests in 0.024s

OK


### DataLoaders

To train neural networks efficiently, a common practice is to batch
multiple samples together to form a mini-batch. Batching fixed-shaped tensor
inputs is common. For example, batching two images of size 28 x 28
gives a tensor of shape 2 x 28 x 28. By contrast, batching graph inputs
has two challenges:

* Graphs are sparse.
* Graphs can have various length. For example, number of nodes and edges.

To address this, DGL provides a :func:`dgl.batch` API. It leverages the idea that
a batch of graphs can be viewed as a large graph that has many disjointed
connected components. Below is a visualization that gives the general idea.

![](https://data.dgl.ai/tutorial/batch/batch.png)

In [ ]:
def collate_graph_batch(samples: List[Tuple[DGLGraph, int]]) -> Tuple[Tensor, Tensor]:
    graphs, labels = map(list, zip(*samples))
    batched_graph = construct_graph_batch(graphs)
    return batched_graph, Tensor(labels)

In [ ]:
BATCH_SIZE = 64
NUM_CLASSES = 8

train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_graph_batch, drop_last=True)
val_dataloader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_graph_batch)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_graph_batch)

## Model implementation

Graph classification proceeds as follows.

![](https://data.dgl.ai/tutorial/batch/graph_classifier.png)


From a batch of graphs, perform message passing and graph convolution for nodes to communicate with others. After message passing, compute a tensor for graph representation from node (and edge) attributes. This step might  be called readout or aggregation. Finally, the graph  representations are fed into a classifier $g$ to predict the graph labels.

Graph convolution layer can be found in the ``dgl.nn.<backend>`` submodule.

In this lab the easiest choice is `GraphConv`. [Docs](https://docs.dgl.ai/en/1.1.x/generated/dgl.nn.pytorch.conv.GraphConv.html#dgl.nn.pytorch.conv.GraphConv).


In [ ]:
class GraphClassifier(nn.Module):
    def __init__(self, hidden_dim, n_classes):
        super().__init__()

        input_dim = 1
        # NOTE: for educational purposes here we use 1 as input dimension.
        # However, in production feature vector is the information about the node.
        # For example, in social networks the feature vector could represent the user
        # (e.g. its choices of movies)
        self.conv1 = GraphConv(input_dim, hidden_dim)
        self.relu1 = nn.ReLU()
        self.conv2 = GraphConv(hidden_dim, hidden_dim)
        self.relu2 = nn.ReLU()
        self.head = nn.Linear(hidden_dim, n_classes)

    def forward(self, graphs):
        # Use node degree as the initial node feature. For undirected graphs, the in-degree
        # is the same as the out_degree.
        h = graphs.in_degrees().view(-1, 1).float()

        # Perform graph convolution and activation function.
        h = self.conv1(graphs, h)
        h = self.relu1(h)
        h = self.conv2(graphs, h)
        h = self.relu2(h)

        graphs.ndata['h'] = h
        # Calculate graph representation by averaging all the node representations.
        hg = dgl.mean_nodes(graphs, 'h')
        return self.head(hg)

## Training


# Exercise 2: Implement Training and Evaluation Loop for Graph Classification Model

## Objective
In this exercise, you will implement a comprehensive training and evaluation loop for a graph classification model using PyTorch. This will involve training the model, validating its performance on a separate dataset, and calculating important performance metrics.

## Instructions

### Setup
1. **Training**: Write a loop to train the model for a specified number of epochs.
2. **Validation**: Implement validation within each epoch to evaluate the model's performance on a validation dataset.
3. **Metrics Calculation**: Calculate and print performance metrics including precision, recall, F1 score, and accuracy.

### Tasks
1. **Progress Bar**: Utilize a progress bar to monitor the training loss during each epoch using `tqdm`.
2. **Device Handling**: Ensure that both data and model are moved to the appropriate device (CPU or GPU).
3. **Memory Management**: Implement memory management to clear GPU cache after each epoch if using CUDA.
4. **Metrics**: Calculate and print the average precision, recall, F1 score, and accuracy after each epoch.

## Deliverables

### Training Loop
- Implement the training loop to:
  - Set the model to training mode.
  - Iterate through the training data, perform a forward pass, calculate the loss, perform backpropagation, and update the model parameters.
  - Track and print the average training loss for each epoch.

### Validation Loop
- Implement the validation loop to:
  - Set the model to evaluation mode.
  - Iterate through the validation data, perform a forward pass, and calculate validation loss.
  - Calculate and print precision, recall, F1 score, and accuracy for the validation set.

### Code Requirements
- Handle different devices (CUDA or CPU).
- Use `tqdm` for progress tracking.
- Clear GPU memory if using CUDA.
- Ensure that metrics are calculated and printed correctly.


In [ ]:
HIDDEN_DIM = 256

# Select the where to perform
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
# Create an instance of the model and pass its weights to the device
model = GraphClassifier(HIDDEN_DIM, NUM_CLASSES).to(device)
# Set the loss function
loss_function = nn.CrossEntropyLoss()
# Set the opimizer
optimizer = torch.optim.Adam(model.parameters(), lr=3e-4) # Using Karpathy's learning rate constant

Here are utility functions to calculate and show metrics:

In [ ]:
def calculate_metric(metric_fn, true_y, pred_y):
    if metric_fn != accuracy_score:
        return metric_fn(true_y, pred_y, average="macro")
    else:
        return metric_fn(true_y, pred_y)

def print_scores(p, r, f1, a, batch_size):
    for name, scores in zip(("precision", "recall", "F1", "accuracy"), (p, r, f1, a)):
        print(f"\t{name.rjust(14, ' ')}: {sum(scores)/batch_size:.4f}")

In [ ]:
epochs = 60

losses = []
batches = len(train_dataloader)
val_batches = len(val_dataloader)

# loop for every epoch (training + evaluation)
for epoch in range(epochs):
    total_loss = 0

    # progress bar
    progress = tqdm(enumerate(train_dataloader), desc="Loss: ", total=batches)

    # ----------------- TRAINING  --------------------
    # set model to training
    model.train()

    #type code here

    # ----------------- VALIDATION  -----------------

    #type code here


Loss:   0%|          | 0/10 [00:00<?, ?it/s]

Epoch 1/60, training loss: 126.2022315979004, validation loss: 121.2534408569336
	     precision: 0.0176
	        recall: 0.1250
	            F1: 0.0306
	      accuracy: 0.1406


Loss:   0%|          | 0/10 [00:00<?, ?it/s]

Epoch 2/60, training loss: 121.16864776611328, validation loss: 120.69944763183594
	     precision: 0.0242
	        recall: 0.1250
	            F1: 0.0401
	      accuracy: 0.1406


Loss:   0%|          | 0/10 [00:00<?, ?it/s]

...

Epoch 58/60, training loss: 54.94828224182129, validation loss: 54.11578369140625
	     precision: 0.8755
	        recall: 0.8550
	            F1: 0.8397
	      accuracy: 0.8542


Loss:   0%|          | 0/10 [00:00<?, ?it/s]

Epoch 59/60, training loss: 53.85807342529297, validation loss: 53.71480941772461
	     precision: 0.8792
	        recall: 0.8653
	            F1: 0.8520
	      accuracy: 0.8698


Loss:   0%|          | 0/10 [00:00<?, ?it/s]

Epoch 60/60, training loss: 53.12934913635254, validation loss: 52.05900573730469
	     precision: 0.7178
	        recall: 0.7765
	            F1: 0.7195
	      accuracy: 0.7500


# Exercise 3: Model Evaluation and Results Generation

## Objective
In this exercise, you will evaluate the performance of a graph classification model on a test dataset and generate a results file. This involves making predictions with the trained model, validating the output, and saving the results for further analysis.

## Instructions

### Tasks

1. **Model Evaluation**:
   - Use the trained model to make predictions on the test dataset.
   - Ensure that the model is set to evaluation mode and that gradients are not calculated during inference to save memory.

2. **Assertions**:
   - Validate the shape of the model's output.
   - Verify the length of the predictions to ensure it matches the expected number of test samples.

3. **Results Generation**:
   - Create a DataFrame to store the test results, including `graph_id` and the corresponding predicted labels.
   - Save the DataFrame to a CSV file named `results.csv`.

### Deliverables

1. **Model Evaluation Code**:
   - Implement code to evaluate the model on the test dataset, making predictions for each graph.

2. **Assertions**:
   - Add assertions to check:
     - The shape of the model’s output.
     - The length of the predictions list.

3. **Results File**:
   - Generate a results file that contains two columns: `graph_id` and `label`.
   - Save the results to `results.csv`.

In [ ]:
predictions = []

# type code here

In [ ]:

# type code here

Test cases passed


In [ ]:
# generate the results file

# type code here